# 基于Gremlin的交互式查询
Gremlin 是图数据领域最流行的查询语言之一，就好比关系型数据库里的 SQL 一样。接下来，我们将通过一些例子来说明 Gremlin 是如何执行图查询的。
## 数据集
MODERN，tinkerpop提供的示例数据集，由6个顶点和6条边组成。通过在 MODERN 基础上加入更多的边，方便我们展示更多的 gremlin 查询语句。这是所有边：
[(1,3),(1,2),(1,4),(4,5),(4,3),(6,3),(2,4),(4,1)]

In [1]:
import os
import graphscope
from graphscope.framework.graph import Graph
from graphscope.framework.loader import Loader
import vineyard

k8s_volumes = {
    "data": {
        "type": "hostPath",
        "field": {
          "path": "/home/jovyan/datasets/modern_graph_2",  # Path in host
          "type": "Directory"
        },
        "mounts": {
          "mountPath": "/home/jovyan/datasets/modern_graph_2"  # Path in pods
          "readOnly": True
        }
    }
}
graphscope.set_option(show_log=True)  # enable logging
session = graphscope.session(k8s_volumes=k8s_volumes)  # create a session
modern_graph = session.load_from(
    vertices={
        "person": (Loader("/home/jovyan/datasets/modern_graph_2/person.csv", delimiter="|", header_row=True), ["name", ("age", "int")], "id"),
        "software": (Loader("/home/jovyan/datasets/modern_graph_2/software.csv", delimiter="|", header_row=True), ["name", "lang"], "id"),
    },
    edges={
        "knows": [Loader("/home/jovyan/datasets/modern_graph_2/knows.csv", delimiter="|"), [], (0, "person"), (1, "person")],
        "created": [Loader("/home/jovyan/datasets/modern_graph_2/created.csv", delimiter="|"), [], (0, "person"), (1, "software")],
    }, generate_eid=False)
interactive = session.gremlin(modern_graph)

2021-01-31 05:32:01,430 [INFO][session:377]: Initializing graphscope session with parameters: {'num_workers': 2, 'k8s_namespace': None, 'k8s_service_type': 'NodePort', 'k8s_gs_image': 'registry.cn-hongkong.aliyuncs.com/graphscope/graphscope:0.1.2', 'k8s_etcd_image': 'quay.io/coreos/etcd:v3.4.13', 'k8s_image_pull_policy': 'IfNotPresent', 'k8s_image_pull_secrets': [], 'k8s_gie_graph_manager_image': 'registry.cn-hongkong.aliyuncs.com/graphscope/maxgraph_standalone_manager:0.1.2', 'k8s_zookeeper_image': 'zookeeper:3.4.14', 'k8s_coordinator_cpu': 1.0, 'k8s_coordinator_mem': '4Gi', 'k8s_etcd_cpu': 0.5, 'k8s_etcd_mem': '128Mi', 'k8s_zookeeper_cpu': 0.5, 'k8s_zookeeper_mem': '128Mi', 'k8s_gie_graph_manager_cpu': 1.0, 'k8s_gie_graph_manager_mem': '4Gi', 'k8s_vineyard_cpu': 0.5, 'k8s_vineyard_mem': '512Mi', 'k8s_vineyard_shared_mem': '4Gi', 'k8s_engine_cpu': 0.5, 'k8s_engine_mem': '4Gi', 'k8s_volumes': {'data': {'type': 'hostPath', 'field': {'path': '/home/jovyan/datasets/modern_graph_2', 'type'

## 两点之间的遍历
两点之间的遍历是图数据领域十分常见的查询场景。例如，为了搞清楚v1和v2/v3之间的关系，我们可以这样写一条 gremlin 语句：

In [2]:
q1 = interactive.execute("g.V().has(\"id\", 1).as(\"u\").out().has(\"id\", eq(2).or(eq(3))).as(\"v\").select(\"u\", \"v\").by(\"id\")")
for p in q1:
    print(p)

[{'u': 1, 'v': 3}, {'u': 1, 'v': 2}]


接下来展示一条在社交网络场景中常用的查询，例如如何找到两个不同用户之间的共同特点，用户1昵称为"marko"，另一个为"peter"。

In [3]:
q2 = interactive.execute("g.V().has(\"name\", \"marko\").out().where(__.in().has(\"name\", \"peter\")).valueMap()")
for p in q2:
    print(p)

[{'name': ['lop'], 'lang': ['java'], 'id': [3]}]


## 度的中心性
度的中心性是衡量每个顶点邻接边数量的指标，在处理大数据时有重要意义，这里有一些示例：

In [4]:
q3 = interactive.execute("g.V().group().by().by(bothE().count())")
for p in q3:
    print(p[0])
q4 = interactive.execute("g.V().group().by().by(inE().count())")
for p in q4:
    print(p[0])

{v[0.0]: 5, v[0.1]: 2, v[1.-4611686018427387904]: 3, v[1.-4611686018427387903]: 1, v[0.2]: 1, v[0.-9223372036854775808]: 4}
{v[0.0]: 2, v[1.-4611686018427387904]: 3, v[0.1]: 1, v[0.-9223372036854775808]: 1, v[0.2]: 0, v[1.-4611686018427387903]: 1}


## 环的检测
环的检测是图查询在商业领域中的另一重要应用，环通常被认为是欺诈行为的发生。这里有一个示例展示 gremlin 如何被用于发现图上的环。

In [5]:
q5 = interactive.execute("g.V().as(\"u\").repeat(out().simplePath()).times(2).where(out().where(eq(\"u\"))).count()")
print(q5.one())

[3]
